In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
# Determine the locations of auxiliary libraries and datasets.
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/Lenta Hack/hack_data")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.insert(0, str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install -U pyarrow

     |████████████████████████████████| 17.3MB 1.4MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
data = pd.read_parquet(f"{AUX_DATA_ROOT}/transactions.parquet")
data = data.sample(frac=1)
data = data[:3000000]

In [ ]:
data

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
20984412,a86413d37475c21a1fe56519f04d689b,26ffcef53c44522efbfe7fef964a4058,2017-05-31,002,c2ef2daf4eab9164bb3495824d358a24,407f0f050aa03707eb57c2450efa3218,1.000,34.89,1
18042080,72e259787d55317c889b899a87385fa9,c9df945f744f1f5927f66cf11a8a990f,2017-05-25,009,1435e3b22c2c8de025c5f7b7f48223ed,5db2df15504d368af6d5b29f660632f2,1.000,22.99,0
16638751,e123dc37778369b45f0bef90d44dd7f8,29150bb2319c182c944841c74d2f8d75,2017-04-01,002,62f9941a2b84d62edb007419066a1dd2,142d63107737a448dc2dfe2dcd0154b8,2.000,13.98,0
24024025,b9cb483b802e5eb42494a4818e8299e1,a8d48ed01e06015083c360366501e726,2017-07-02,010,59e5301a72580f94081700bd227821a1,2c8e198c97bb15305aec2f9ef92424ae,1.000,39.89,1
14684936,9b81a9632ba9144946f7db8f38fc767f,09bb9b610c9bdc2393e852c23df1ccf3,2017-03-16,004,7b15d6aeac45fe9ae8fd5a107c982371,5a65adf801e108c242f799719f570caf,1.000,14.49,0
...,...,...,...,...,...,...,...,...,...
15717380,8d9188da3f2dc2e271df03f9d039c4aa,d1f06d78c49ae2f50ed110bf71b14d5d,2017-03-29,001,d0b0c2e731ece56156266d5a9c4ab03e,14f9669f6e03e603124e0831bd7efaa8,1.000,137.89,1
15970075,51ee467ab57f7bd8062762e5b7bd05c8,db2f40f24260bc41db48d82d5e7abf1d,2017-10-04,0000000011,9ff02748166d36242edb2f4546f528b4,265f8fb8751aa4c1b8d91323addac308,1.000,46.99,0
18810879,07bbb043815188824bcd2310a09eb879,68a7e85c6872a4685c2e7a29ba330461,2017-04-24,011,64fbf43e7c3806d7ee523dfcaabc9fb8,1cbaaeb2212bbbe9d458ca9b81d74db7,1.378,89.42,0
18217889,0fba28c8ee3e4754a8d1cc3324aa4823,6e746846ad849391755c57db7166827c,2017-04-14,003,3c33d2c90b5f00f865b83f8a82080448,cfd9f4e33120a3b8870c477b4df46e63,2.000,274.38,0


In [ ]:
data['price'] = data['sales_sum']/data['sales_count']
data

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo,price
20984412,a86413d37475c21a1fe56519f04d689b,26ffcef53c44522efbfe7fef964a4058,2017-05-31,002,c2ef2daf4eab9164bb3495824d358a24,407f0f050aa03707eb57c2450efa3218,1.000,34.89,1,34.890000
18042080,72e259787d55317c889b899a87385fa9,c9df945f744f1f5927f66cf11a8a990f,2017-05-25,009,1435e3b22c2c8de025c5f7b7f48223ed,5db2df15504d368af6d5b29f660632f2,1.000,22.99,0,22.990000
16638751,e123dc37778369b45f0bef90d44dd7f8,29150bb2319c182c944841c74d2f8d75,2017-04-01,002,62f9941a2b84d62edb007419066a1dd2,142d63107737a448dc2dfe2dcd0154b8,2.000,13.98,0,6.990000
24024025,b9cb483b802e5eb42494a4818e8299e1,a8d48ed01e06015083c360366501e726,2017-07-02,010,59e5301a72580f94081700bd227821a1,2c8e198c97bb15305aec2f9ef92424ae,1.000,39.89,1,39.890000
14684936,9b81a9632ba9144946f7db8f38fc767f,09bb9b610c9bdc2393e852c23df1ccf3,2017-03-16,004,7b15d6aeac45fe9ae8fd5a107c982371,5a65adf801e108c242f799719f570caf,1.000,14.49,0,14.490000
...,...,...,...,...,...,...,...,...,...,...
15717380,8d9188da3f2dc2e271df03f9d039c4aa,d1f06d78c49ae2f50ed110bf71b14d5d,2017-03-29,001,d0b0c2e731ece56156266d5a9c4ab03e,14f9669f6e03e603124e0831bd7efaa8,1.000,137.89,1,137.890000
15970075,51ee467ab57f7bd8062762e5b7bd05c8,db2f40f24260bc41db48d82d5e7abf1d,2017-10-04,0000000011,9ff02748166d36242edb2f4546f528b4,265f8fb8751aa4c1b8d91323addac308,1.000,46.99,0,46.990000
18810879,07bbb043815188824bcd2310a09eb879,68a7e85c6872a4685c2e7a29ba330461,2017-04-24,011,64fbf43e7c3806d7ee523dfcaabc9fb8,1cbaaeb2212bbbe9d458ca9b81d74db7,1.378,89.42,0,64.891147
18217889,0fba28c8ee3e4754a8d1cc3324aa4823,6e746846ad849391755c57db7166827c,2017-04-14,003,3c33d2c90b5f00f865b83f8a82080448,cfd9f4e33120a3b8870c477b4df46e63,2.000,274.38,0,137.190000


In [ ]:
# clients_path = f"{AUX_DATA_ROOT}/clients.csv"
# clients = pd.read_csv(clients_path)

materials_path = f"{AUX_DATA_ROOT}/materials.csv"
materials = pd.read_csv(materials_path)

# plants_path = f"{AUX_DATA_ROOT}/plants.csv"
# plants = pd.read_csv(plants_path)

In [ ]:
materials

,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,35cbdf61de9e19c8b417327aaef14c88,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,9eed45f71360b4b1e2590637467220e5,212a38db0ddcd009f1e164cc8483485c,0,0
1,c0b0bf24d4ec71da3d304f761ec555d8,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,de2eb747e0896c050905a6b635ab800a,9b0b6c7d55413ad3b67761b7b125b534,0,0
2,f0fc5e654a81a7c4b8ba8d7c26546e14,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,8a969031832c535daf96e0c2aed8e814,9b0b6c7d55413ad3b67761b7b125b534,0,0
3,353693e64fb5f9e2d29746d7fe6edf1e,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
4,5d9e0b4302ce95448cae72165ff4cf5b,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
...,...,...,...,...,...,...,...,...
105604,feeddbacd9eed3bac85b1571ac79e40e,NONFOOD,3295c76acbf4caaed33c36b1b5fc2cb1,11704817e347269b7254e744b5e22dac,e47e165040e2914783a451509bbc8180,8ca0d8745f91fb71551be26b1ca854f4,0,0
105605,e218c4c3676927e53b95a12ffe9e721a,FOOD,02e74f10e0327ad868d138f2b4fdd6f0,df0e09d6f25a15a815563df9827f48fa,a03ddfb4b641e98de3223765162217c3,e3319f2f4767af113d82174f662f2d4e,1,1
105606,9a5e2e34d8406616f5981868b4576444,NONFOOD,fc490ca45c00b1249bbe3554a4fdf6fb,865bf46435bd84fa5d89f64cf3ba7347,e2a75db437bee7d34615fccf32d7a464,546614030cdbddcae3de351852962759,0,0
105607,d208910eb82fa1f57684f793f93521a4,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,90365351ccc7437a1309dc64e4db32a3,ea9cffc9bd2bf8a0fb128d223f6a4282,7f1dbcf6ab15c23093508627ca12b8b1,0,0


In [ ]:
data = pd.merge(data, materials, how = 'left', on='material')
data

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo,price,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,a86413d37475c21a1fe56519f04d689b,26ffcef53c44522efbfe7fef964a4058,2017-05-31,002,c2ef2daf4eab9164bb3495824d358a24,407f0f050aa03707eb57c2450efa3218,1.000,34.89,1,34.890000,FOOD,6c8349cc7260ae62e3b1396831a8398f,767b2cc82cecc0385fe6f1086dd2c748,025e97d0604fac5f9b246cba3f89a378,fecaff9801e3c7f766618422d1154957,0.0,0.0
1,72e259787d55317c889b899a87385fa9,c9df945f744f1f5927f66cf11a8a990f,2017-05-25,009,1435e3b22c2c8de025c5f7b7f48223ed,5db2df15504d368af6d5b29f660632f2,1.000,22.99,0,22.990000,FOOD,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,130d817d8b3f616e97c555ec57946aec,a3d2de7675556553a5f08e4c88d2c228,0.0,0.0
2,e123dc37778369b45f0bef90d44dd7f8,29150bb2319c182c944841c74d2f8d75,2017-04-01,002,62f9941a2b84d62edb007419066a1dd2,142d63107737a448dc2dfe2dcd0154b8,2.000,13.98,0,6.990000,NONFOOD,d645920e395fedad7bbbed0eca3fe2e0,bd3ef5c19067fe179f71c8b86ea4b39b,5bf563e8f99ed820f322704e4898df6b,b57d38d46f9915a4d2752d53d5cf70e0,1.0,0.0
3,b9cb483b802e5eb42494a4818e8299e1,a8d48ed01e06015083c360366501e726,2017-07-02,010,59e5301a72580f94081700bd227821a1,2c8e198c97bb15305aec2f9ef92424ae,1.000,39.89,1,39.890000,FOOD,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76,f6d077b89fde801cd8031bb3cbbaa8a9,0.0,0.0
4,9b81a9632ba9144946f7db8f38fc767f,09bb9b610c9bdc2393e852c23df1ccf3,2017-03-16,004,7b15d6aeac45fe9ae8fd5a107c982371,5a65adf801e108c242f799719f570caf,1.000,14.49,0,14.490000,FOOD,17e62166fc8586dfa4d1bc0e1742c08b,b90ba83119860d7f6a6dfaab9f2aa150,9b265fdfc7880e370c421b7f1c1f2d70,195e993ae60d31187f42cc9a509d8a22,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,8d9188da3f2dc2e271df03f9d039c4aa,d1f06d78c49ae2f50ed110bf71b14d5d,2017-03-29,001,d0b0c2e731ece56156266d5a9c4ab03e,14f9669f6e03e603124e0831bd7efaa8,1.000,137.89,1,137.890000,FOOD,182be0c5cdcd5072bb1864cdee4d3d6e,8217bb4e7fa0541e0f5e04fea764ab91,470152be04c675460087661ddc61ef5a,3cdd35521f057d08781df87795f2eb19,0.0,0.0
2999996,51ee467ab57f7bd8062762e5b7bd05c8,db2f40f24260bc41db48d82d5e7abf1d,2017-10-04,0000000011,9ff02748166d36242edb2f4546f528b4,265f8fb8751aa4c1b8d91323addac308,1.000,46.99,0,46.990000,FOOD,6c8349cc7260ae62e3b1396831a8398f,09a69de15cf89bc7fe8c0642f906a4dd,dd1f6b0d695ea9f97da90b5cfd8fed06,a2bc51a6d6ccd2d1c0e7992e1812b9e8,0.0,0.0
2999997,07bbb043815188824bcd2310a09eb879,68a7e85c6872a4685c2e7a29ba330461,2017-04-24,011,64fbf43e7c3806d7ee523dfcaabc9fb8,1cbaaeb2212bbbe9d458ca9b81d74db7,1.378,89.42,0,64.891147,FOOD,6f4922f45568161a8cdf4ad2299f6d23,645098b086d2f9e1e0e939c27f9f2d6f,d66a5517fea10ab5d81ee32912ef07e6,3e9e3d22f8052f4eb926f2f82c226ed2,0.0,0.0
2999998,0fba28c8ee3e4754a8d1cc3324aa4823,6e746846ad849391755c57db7166827c,2017-04-14,003,3c33d2c90b5f00f865b83f8a82080448,cfd9f4e33120a3b8870c477b4df46e63,2.000,274.38,0,137.190000,FOOD,1c383cd30b7c298ab50293adfecb7b18,2a3d6d6cc4b5e77238c1fc1bb6cdd681,cd7f6cf8614708311ba03c03102f8910,60cd9ce5a6420acb1f6802fa746676cd,0.0,0.0


In [ ]:
freq = data.groupby(['client_id', 'hier_level_3', 'hier_level_4', 'material']).chq_id.count().reset_index().sort_values('chq_id',ascending=False).set_index('client_id')
freq.rename(columns = {"chq_id":"frequency"}, inplace = True)

In [ ]:
freq.head(40)

,hier_level_3,hier_level_4,material,frequency
client_id,,,,
dfb684ff08c6dea880d5eb9f0e9309cf,a3788c8c64fd65c470e23e7534c3ebc8,29a90297dcc11ab043a1ffb971246c08,c6571b7b1b7ce43041791bd91a5a65dd,114
35e38d87f20cead704467ba83c0f0ec3,7501e5d4da87ac39d782741cd794002d,9076db4895b417d4a668cb0b7168e4db,c8ac520562ba934dd67cbf0f060c4252,107
ac50f6d558766dceed03dae195d0d151,2eb5657d37f474e4c4cf01e4882b8962,007496ff404d0a1d4e5c993492d4b99a,6964e7ca85a4815c0c9dd8255c7bcff4,95
d316c6fb1ecfc3303ef286ad7426a642,7501e5d4da87ac39d782741cd794002d,9076db4895b417d4a668cb0b7168e4db,c8ac520562ba934dd67cbf0f060c4252,84
cd2a8f57759a0d247db058321b6a12b3,a3788c8c64fd65c470e23e7534c3ebc8,29a90297dcc11ab043a1ffb971246c08,d99ff1bb6ab63242de3b682b51461dce,76
0b1b96c791f5779ad006171187b2d31e,7501e5d4da87ac39d782741cd794002d,9076db4895b417d4a668cb0b7168e4db,c8ac520562ba934dd67cbf0f060c4252,72
4341c3b1a1c9d78a7c27e4c11ec7eb2f,f2d887e01a80e813d9080038decbbabb,96e0bececffda323c59ab5ff3751bf76,6278f48a43fb0a23036b3a11c92e726c,47
a730942ae3157d688a9b96180c3048d3,64c31821603ab476a318839606743bd6,6c1159bbba2c22e4948ebd81ec5932dd,00dd66971e21a68bb052882103bec2a1,46
ef0bb7c50945fd32967badc6f7e3134e,7d265aa7147bd3913fb84c7963a209d1,65ce9a05680f7ff3339a68cc1eac2330,82db77b19a1d3b622fedf5b1f7dbc1ed,45


In [ ]:
promo = data.groupby(['client_id', 'hier_level_3', 'hier_level_4', 'material']).agg({'is_promo': 'sum'})
promo

is_promo
client_id                        hier_level_3                     hier_level_4                     material                                  
0000d48a8891cd462fdf01ea861ec53e 077b83af57538aa183971a2fe0971ec1 fd32e508db711e1879edd10f1820ddc7 b3f18b3fef2e0b1a6f35cb7cc4298c51         0
                                 09a69de15cf89bc7fe8c0642f906a4dd fb900ec217f36927c7ba652042148331 1900cafcab3e9dafe33927178f17b6b8         1
                                 124461dcd3571e6674ec4e0e140cc298 ac86f5ba1983d6a4860d0c22ee2a7679 5041e78160873dd97377183c917b630a         1
                                 149ef6419512be56a93169cd5e6fa8fd d9b545c7ce73410f8c9f4f7bbd429717 da163f4dfbea1578d9b64616e35bf66d         0
                                 24ec8468b67314c2013d215b77034476 f4e65cd8421074ce1c3ef34d181ad4fa a7edd6c7354237c10469e53e62cbc3e3         1
...                                                                                                                                       ...
fffc477269ae5f219e39aa51fa7abb81 e71e5cd119bbc5797164fb0cd7fd94a4 91534231d73afa8f5765e15d83711a52 6c54ca6fd28928b73e9b6c3e796c68b7         1
                                 f23b3df742bb9fbf6bbf30a05150ac19 19581aa371a687823cd7840e58087492 bdfe297c862890113e7fd79922e0bc27         1
fffd3e78e3ea6bf00aee1372efc09dad 155fa09596c7e18e50b58eb7e0c6ccb4 8f4d4869ec6fc30d05ad643b952e0dbc 5853283dece1f7a4da50cc4bebd5b116         1
                                 3a61ed715ee66c48bacf237fa7bb5289 22aef2a6748ffb4853c2e0a9127654d1 3e9e654e56cb4c5c4631b256c5d5753e         0
                                 645098b086d2f9e1e0e939c27f9f2d6f d66a5517fea10ab5d81ee32912ef07e6 f27d1e86a3fdddc3efea8d9b460376aa         1

[2659698 rows x 1 columns]

In [ ]:
freq_promo = pd.merge(freq, promo, how = 'left', on=['client_id', 'hier_level_3', 'hier_level_4', 'material'])
freq_promo['price'] = freq_promo[['material']].apply(lambda x : data.query('material == "%s"' % x[0]).price.mean(), axis=1)

In [ ]:
freq_promo.reset_index(inplace=True)

## Customer Categorization

In [ ]:
def is_article_oriented(row):
  client = row.client_id
  material = row.material
  freq = row.frequency
  price = row.price
  level = row.hier_level_4

  discounts = True
  cheap = True

  client_table = freq_promo.loc[((freq_promo.client_id == client) & (freq_promo.material == material))]

  if (len(client_table[client_table.is_promo == 0]) > 0): # The client buys the article without a discount even for one time
    discounts = False
  
  materials_table = freq_promo[(freq_promo.hier_level_4 == level)]
  if (materials_table.price.mean() < price): # The client buys the most expensive material
    cheap = False

  return not (discounts or cheap)


freq_promo['is_article_oriented'] = freq_promo.apply(is_article_oriented, axis=1)
freq_promo.head()

In [ ]:
print(freq_promo[freq_promo.is_article_oriented == True].shape[0] / freq_promo.shape[0])

## Article Scoring

In [ ]:
def has_negative_feedback(client, material):
  q = data.query('client_id == "%s" and material == "%s"' % (client, material))
  return len(q[q.return_count >= 1]) > 0  #Has a negative feedback
def compute_score(row):
  client = row.client_id
  material = row.material
  level = row.hier_level_4
  freq = row.frequency
  score = 0
  if not has_negative_feedback(client, material):
    score = freq / freq_promo.query('client_id == "%s" and hier_level_4 == "%s"' % (client, level)).frequency.sum()
  
  return score

freq_promo['material_score'] = freq_promo.apply(compute_score, axis=1)
freq_promo.head()

In [ ]:
freq_promo[freq_promo.material_score < 1]

In [ ]:
freq_promo[((freq_promo.hier_level_4 == '65ce9a05680f7ff3339a68cc1eac2330') & (freq_promo.client_id == '759165e34fbdc27550ccf84ceeefac17'))]

## Data split

In [ ]:
price_oriented_people = freq_promo[~ freq_promo.is_article_oriented].copy()
article_oriented_people = freq_promo[freq_promo.is_article_oriented].copy()

In [ ]:
price_oriented_people.head()

In [ ]:
article_oriented_people.head()

#AO Score vs. PO Score (Article Oriented Vs. product Oriented)


In [ ]:
def compute_total_score(row, po=True):
  material = row.material
  if po:
   return price_oriented_people[price_oriented_people.material == material].material_score.sum()

  else:
    return article_oriented_people[article_oriented_people.material == material].material_score.sum()


In [ ]:
price_oriented_people['PO_Score'] = price_oriented_people.apply(compute_total_score, axis=1)

In [ ]:
price_oriented_people.head()

In [ ]:
article_oriented_people['AO_Score'] = article_oriented_people[['material']].apply(compute_total_score, axis=1, args=(False,))

In [ ]:
article_oriented_people.head()

# FIND PRICES FOR MATERIALS 

In [ ]:
df1 = data[['client_id', 'price', 'hier_level_3', 'hier_level_4', 'material', 'sales_count']].copy()
def compute_count(x):
  if x.sales_count < 0:
    return 1
  else:
    return 0
df1['return_count'] = df1.apply(compute_count, axis=1)
df1 = df1.groupby(['hier_level_3', 'hier_level_4', 'material']).agg({'price': 'mean', 'return_count': 'sum'} , axis = 0).reset_index()
df1

In [ ]:
df1['material'].count()

In [ ]:
data.drop('price' , axis = 1, inplace = True, errors='ignore')

In [ ]:
def price_for_material(x):
  return x.price
data['price'] = df1.apply(price_for_material, axis = 1)
data

In [ ]:
df2 = data[['client_id', 'price', 'hier_level_3', 'hier_level_4', 'material', 'sales_count']].copy()
def compute_count(x):
  if x.sales_count < 0:
    return 1
  else:
    return 0
df2['return_count'] = df2.apply(compute_count, axis=1)
df2 = df2.groupby(['client_id', 'hier_level_3', 'hier_level_4', 'material']).agg({'price': 'mean', 'return_count': 'sum'} , axis = 0).reset_index()
df2

# FINAL TABLE

# FREQ_PROMO_PRICE

In [ ]:
# freq_promo_price = pd.merge(freq_promo, df1, how = 'left', on=['client_id','hier_level_3', 'hier_level_4','material'])
# freq_promo_price = freq_promo_price.drop('sales_count', axis = 1)
# freq_promo_price

In [ ]:
#freq_promo_price.head(40)